In [2]:
import numpy as np 
import pandas as pd 
import os
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler 
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold  
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler 
import lightgbm as lgbm
from catboost import CatBoostClassifier 
from xgboost import XGBClassifier

# import pycaret library for binary classification 
from pycaret.classification import *
import pycaret

In [3]:
#setup train and test dataframes 
train = pd.read_csv('train.csv') 
test = pd.read_csv('test_x.csv') 
submission = pd.read_csv('sample_submission.csv') 

train = train.drop(columns=['index']) 
test = test.drop(columns=['index'])

In [4]:
clf = setup(data = train, target = 'voted')

Setup Succesfully Completed!


,Description,Value
0,session_id,3899
1,Target Type,Binary
2,Label Encoded,"1: 0, 2: 1"
3,Original Data,"(45532, 77)"
4,Missing Values,False
5,Numeric Features,41
6,Categorical Features,35
7,Ordinal Features,False
8,High Cardinality Features,False
9,High Cardinality Method,None


# Create LGBM

In [5]:
lgbm = create_model('lightgbm')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.6866,0.7511,0.6477,0.7457,0.6933,0.3762,0.3801
1,0.6851,0.7580,0.6500,0.7420,0.6930,0.3726,0.3760
2,0.6909,0.7667,0.6527,0.7495,0.6978,0.3846,0.3885
3,0.7013,0.7659,0.6429,0.7724,0.7018,0.4075,0.4144
4,0.6941,0.7598,0.6558,0.7530,0.7010,0.3908,0.3948
5,0.6840,0.7601,0.6225,0.7566,0.6830,0.3739,0.3810
6,0.6878,0.7615,0.6357,0.7548,0.6901,0.3802,0.3859
7,0.7035,0.7704,0.6328,0.7834,0.7001,0.4134,0.4226
8,0.6975,0.7625,0.6368,0.7703,0.6972,0.4004,0.4077
9,0.7057,0.7733,0.6495,0.7759,0.7071,0.4160,0.4226


In [6]:
tuned_lgbm = tune_model(lgbm, n_iter = 100, optimize = 'AUC')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.6848,0.7543,0.6328,0.7514,0.6870,0.3741,0.3797
1,0.6895,0.7602,0.6431,0.7529,0.6937,0.3827,0.3876
2,0.6972,0.7684,0.6521,0.7599,0.7019,0.3979,0.4027
3,0.6972,0.7678,0.6320,0.7726,0.6953,0.4003,0.4083
4,0.7029,0.7576,0.6466,0.7730,0.7042,0.4104,0.4170
5,0.6950,0.7651,0.6317,0.7694,0.6938,0.3958,0.4035
6,0.6796,0.7586,0.6213,0.7500,0.6796,0.3649,0.3713
7,0.7057,0.7755,0.6397,0.7825,0.7039,0.4171,0.4255
8,0.6925,0.7604,0.6311,0.7655,0.6918,0.3906,0.3979
9,0.7038,0.7762,0.6460,0.7749,0.7046,0.4125,0.4193


In [7]:
calibrated_lgbm = calibrate_model(tuned_lgbm)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.6854,0.7537,0.6495,0.7428,0.6930,0.3733,0.3769
1,0.6888,0.7599,0.6615,0.7415,0.6992,0.3791,0.3817
2,0.6947,0.7669,0.6676,0.7469,0.7051,0.3907,0.3933
3,0.6947,0.7680,0.6447,0.7603,0.6977,0.3935,0.3990
4,0.7003,0.7608,0.6581,0.7616,0.7061,0.4038,0.4083
5,0.6944,0.7662,0.6443,0.7603,0.6975,0.3929,0.3984
6,0.6809,0.7599,0.6368,0.7430,0.6858,0.3655,0.3700
7,0.7044,0.7770,0.6546,0.7704,0.7078,0.4127,0.4184
8,0.6906,0.7626,0.6460,0.7532,0.6955,0.3848,0.3895
9,0.7032,0.7758,0.6598,0.7651,0.7086,0.4095,0.4142


In [8]:
predict_holdout = predict_model(calibrated_lgbm)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Calibrated Classifier C V,0.698,0.7685,0.6598,0.7567,0.7049,0.3985,0.4024


# Create Extra Trees Classifier

In [9]:
et = create_model('et') 

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.6819,0.7531,0.6460,0.7393,0.6895,0.3665,0.3700
1,0.6801,0.7546,0.6495,0.7346,0.6894,0.3621,0.3650
2,0.6912,0.7611,0.6498,0.7517,0.6970,0.3856,0.3899
3,0.6994,0.7689,0.6470,0.7667,0.7017,0.4031,0.4090
4,0.6869,0.7566,0.6460,0.7472,0.6929,0.3769,0.3810
5,0.6909,0.7601,0.6317,0.7625,0.6909,0.3873,0.3941
6,0.6815,0.7496,0.6345,0.7453,0.6855,0.3671,0.3720
7,0.7032,0.7658,0.6517,0.7702,0.7060,0.4105,0.4163
8,0.6925,0.7627,0.6368,0.7618,0.6938,0.3899,0.3962
9,0.7022,0.7710,0.6546,0.7668,0.7063,0.4081,0.4134


In [10]:
tuned_et = tune_model(et, n_iter = 100, optimize = 'AUC')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.5994,0.7377,0.9185,0.5852,0.7149,0.1414,0.1899
1,0.6035,0.7419,0.9225,0.5875,0.7179,0.1501,0.2017
2,0.6090,0.7509,0.9328,0.5901,0.7229,0.1611,0.2201
3,0.6072,0.7518,0.9185,0.5904,0.7188,0.1594,0.2098
4,0.5999,0.7462,0.9225,0.5852,0.7161,0.1415,0.1922
5,0.6175,0.7545,0.9254,0.5970,0.7258,0.1816,0.2373
6,0.6134,0.7370,0.9237,0.5943,0.7233,0.1725,0.2267
7,0.6125,0.7612,0.9317,0.5927,0.7245,0.1689,0.2277
8,0.6109,0.7393,0.9157,0.5935,0.7202,0.1682,0.2176
9,0.6094,0.7546,0.9248,0.5913,0.7214,0.1629,0.2171


In [11]:
# et has higher mean AUC than tuned_et 
calibrated_et = calibrate_model(et) 

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.6876,0.7573,0.6604,0.7402,0.6980,0.3766,0.3792
1,0.6810,0.7587,0.6638,0.7286,0.6947,0.3622,0.3639
2,0.6909,0.7663,0.6636,0.7434,0.7012,0.3833,0.3859
3,0.7041,0.7748,0.6579,0.7676,0.7085,0.4117,0.4167
4,0.6997,0.7631,0.6747,0.7510,0.7108,0.4004,0.4029
5,0.7069,0.7724,0.6569,0.7731,0.7103,0.4177,0.4234
6,0.6878,0.7580,0.6443,0.7497,0.6930,0.3791,0.3836
7,0.7057,0.7760,0.6672,0.7646,0.7126,0.4139,0.4179
8,0.6969,0.7651,0.6535,0.7588,0.7022,0.3971,0.4017
9,0.7019,0.7741,0.6655,0.7597,0.7095,0.4062,0.4099


In [12]:
predict_holdout = predict_model(calibrated_et)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Calibrated Classifier C V,0.6982,0.7698,0.6661,0.7534,0.7071,0.3982,0.4014


# Create Random Forest 

In [13]:
rf = create_model('rf') 

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.6518,0.7053,0.5984,0.7178,0.6527,0.3093,0.3145
1,0.6568,0.7159,0.6145,0.7173,0.6619,0.3177,0.3217
2,0.6586,0.7182,0.6045,0.7252,0.6594,0.3228,0.3282
3,0.6605,0.7194,0.6068,0.7270,0.6615,0.3265,0.3319
4,0.6495,0.7056,0.5915,0.7180,0.6486,0.3055,0.3112
5,0.6555,0.7091,0.5995,0.7232,0.6556,0.3169,0.3225
6,0.6454,0.7030,0.5858,0.7145,0.6438,0.2977,0.3035
7,0.6614,0.7182,0.5984,0.7335,0.6591,0.3296,0.3364
8,0.6476,0.7055,0.6013,0.7100,0.6511,0.3001,0.3043
9,0.6683,0.7219,0.6259,0.7293,0.6737,0.3405,0.3446


In [14]:
tuned_rf = tune_model(rf, n_iter = 100, optimize = 'AUC')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.6857,0.7523,0.6363,0.7508,0.6888,0.3757,0.3809
1,0.6763,0.7516,0.6443,0.7316,0.6852,0.3548,0.3578
2,0.6831,0.7577,0.6412,0.7437,0.6887,0.3695,0.3738
3,0.6981,0.7689,0.6418,0.7679,0.6992,0.4011,0.4076
4,0.6919,0.7623,0.6500,0.7528,0.6977,0.3869,0.3913
5,0.6981,0.7629,0.6322,0.7744,0.6961,0.4023,0.4105
6,0.6847,0.7526,0.6294,0.7534,0.6858,0.3744,0.3805
7,0.7038,0.7744,0.6437,0.7765,0.7039,0.4127,0.4200
8,0.6909,0.7553,0.6374,0.7589,0.6929,0.3865,0.3925
9,0.6978,0.7688,0.6443,0.7660,0.6999,0.4002,0.4063


In [15]:
calibrated_rf = calibrate_model(tuned_rf)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.6882,0.7580,0.6558,0.7437,0.6970,0.3785,0.3817
1,0.6813,0.7548,0.6540,0.7341,0.6917,0.3641,0.3667
2,0.6900,0.7682,0.6613,0.7432,0.6999,0.3816,0.3843
3,0.6997,0.7699,0.6458,0.7679,0.7016,0.4039,0.4101
4,0.6969,0.7654,0.6609,0.7544,0.7046,0.3962,0.3998
5,0.6956,0.7713,0.6408,0.7645,0.6973,0.3960,0.4022
6,0.6825,0.7582,0.6328,0.7478,0.6855,0.3693,0.3746
7,0.7016,0.7766,0.6512,0.7679,0.7048,0.4072,0.4129
8,0.6960,0.7625,0.6489,0.7601,0.7001,0.3957,0.4008
9,0.7003,0.7740,0.6609,0.7599,0.7070,0.4034,0.4075


In [16]:
predict_holdout = predict_model(calibrated_rf)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Calibrated Classifier C V,0.6974,0.7684,0.6564,0.7579,0.7035,0.3979,0.4022


# Create CatBoost Classifier 

In [17]:
catboost = create_model('catboost')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.6807,0.7544,0.6586,0.7307,0.6928,0.3622,0.3643
1,0.6826,0.7572,0.6592,0.7332,0.6943,0.3661,0.3684
2,0.6981,0.7688,0.6745,0.7484,0.7095,0.3971,0.3995
3,0.7047,0.7750,0.6567,0.7693,0.7086,0.4131,0.4185
4,0.6960,0.7608,0.6609,0.7529,0.7039,0.3942,0.3977
5,0.6897,0.7676,0.6483,0.7503,0.6956,0.3825,0.3868
6,0.6897,0.7627,0.6529,0.7477,0.6971,0.3820,0.3857
7,0.7041,0.7753,0.6627,0.7649,0.7101,0.4111,0.4155
8,0.6944,0.7606,0.6483,0.7579,0.6988,0.3924,0.3974
9,0.7038,0.7748,0.6592,0.7665,0.7088,0.4109,0.4157


In [19]:
tuned_catboost = tune_model(catboost, n_iter = 100, optimize = 'AUC')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.6885,0.7584,0.6408,0.7527,0.6923,0.3810,0.3861
1,0.6895,0.7598,0.6495,0.7492,0.6958,0.3819,0.3860
2,0.6972,0.7688,0.6533,0.7592,0.7023,0.3977,0.4024
3,0.7000,0.7724,0.6395,0.7725,0.6997,0.4053,0.4126
4,0.7010,0.7632,0.6552,0.7644,0.7056,0.4054,0.4104
5,0.6975,0.7700,0.6305,0.7745,0.6951,0.4012,0.4096
6,0.6884,0.7616,0.6299,0.7593,0.6886,0.3822,0.3889
7,0.7069,0.7793,0.6431,0.7823,0.7059,0.4194,0.4274
8,0.6947,0.7633,0.6265,0.7723,0.6918,0.3958,0.4043
9,0.7003,0.7781,0.6437,0.7706,0.7015,0.4055,0.4121


In [20]:
calibrated_catboost = calibrate_model(tuned_catboost)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.6857,0.7590,0.6546,0.7404,0.6949,0.3733,0.3764
1,0.6876,0.7599,0.6672,0.7365,0.7002,0.3757,0.3777
2,0.6916,0.7700,0.6670,0.7425,0.7028,0.3841,0.3865
3,0.6985,0.7726,0.6533,0.7612,0.7031,0.4004,0.4052
4,0.7010,0.7637,0.6718,0.7545,0.7108,0.4034,0.4063
5,0.6985,0.7702,0.6466,0.7656,0.7011,0.4012,0.4071
6,0.6881,0.7607,0.6460,0.7492,0.6938,0.3795,0.3839
7,0.7107,0.7798,0.6638,0.7749,0.7151,0.4248,0.4301
8,0.6912,0.7634,0.6443,0.7552,0.6954,0.3863,0.3914
9,0.7032,0.7781,0.6604,0.7648,0.7087,0.4094,0.4140


In [21]:
predict_holdout = predict_model(calibrated_catboost)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Calibrated Classifier C V,0.699,0.7714,0.6643,0.7557,0.707,0.4001,0.4036


# Soft Voting 

In [22]:
models = [] 
models.append(calibrated_lgbm)
models.append(calibrated_et)
models.append(calibrated_rf)
models.append(calibrated_catboost) 

In [23]:
blended = blend_models(estimator_list = models, fold = 5, method = 'soft')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.6839,0.7593,0.6586,0.7357,0.6950,0.3691,0.3715
1,0.6944,0.7702,0.6563,0.7531,0.7014,0.3915,0.3954
2,0.6988,0.7689,0.6542,0.7613,0.7037,0.4009,0.4057
3,0.6988,0.7700,0.6514,0.7630,0.7028,0.4013,0.4064
4,0.6988,0.7711,0.6526,0.7624,0.7032,0.4011,0.4061
Mean,0.6949,0.7679,0.6546,0.7551,0.7012,0.3928,0.3970
SD,0.0058,0.0044,0.0026,0.0104,0.0032,0.0124,0.0134


In [24]:
predict_holdout = predict_model(blended) 

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Voting Classifier,0.6985,0.7717,0.6605,0.7571,0.7055,0.3995,0.4034


In [25]:
# train on the rest of the dataset 
final_model = finalize_model(blended) 

In [26]:
predictions = predict_model(final_model, data = test) 

In [27]:
submission['voted'] = predictions['Score']

In [28]:
submission.to_csv('soft_voting_lgbm_et_rf_catboost.csv',index=False)

# Stacking with lgbm meta model 

In [29]:
meta_lgbm = create_model('lightgbm') 
stack_soft_lgbm = stack_models(models, meta_model = meta_lgbm)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.6782,0.7548,0.6351,0.7395,0.6833,0.3600,0.3643
1,0.6873,0.7586,0.6489,0.7460,0.6941,0.3774,0.3812
2,0.7013,0.7677,0.6596,0.7619,0.7071,0.4055,0.4099
3,0.6997,0.7700,0.6349,0.7751,0.6980,0.4052,0.4132
4,0.6890,0.7586,0.6454,0.7510,0.6942,0.3816,0.3861
5,0.6997,0.7677,0.6317,0.7775,0.6971,0.4056,0.4143
6,0.6825,0.7582,0.6259,0.7519,0.6832,0.3702,0.3765
7,0.7007,0.7750,0.6288,0.7812,0.6968,0.4080,0.4174
8,0.6975,0.7661,0.6380,0.7696,0.6976,0.4003,0.4073
9,0.7051,0.7796,0.6460,0.7771,0.7055,0.4151,0.4222


In [30]:
pred_holdout = predict_model(stack_soft_lgbm)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Stacking Classifier,0.6956,0.767,0.6419,0.7638,0.6976,0.3958,0.4019


In [31]:
final_model = finalize_model(stack_soft_lgbm) 

In [36]:
predictions = predict_model(final_model, data = test)  

In [37]:
submission['voted'] = predictions['Score']

In [38]:
submission.to_csv('meta_lgbm_soft_stacking.csv',index=False)

In [40]:
df1 = pd.read_csv('soft_voting_lgbm_et_rf_catboost.csv')
df2 = pd.read_csv('meta_lgbm_soft_stacking.csv')
avg_score = (df1['voted'] + df2['voted'])/2.0 

submission['voted'] = avg_score 

submission.to_csv('intermediate_check.csv',index=False)   

In [41]:
submission.head()

,index,voted
0,0,0.71890
1,1,0.90110
2,2,0.46305
3,3,0.18650
4,4,0.79280


# Saving Models 

In [43]:
save_model(models[0],'lgbm_calibrated') 
save_model(models[1],'et_calibrated')
save_model(models[2],'rf_calibrated') 
save_model(models[3],'catboost_calibrated') 

Transformation Pipeline and Model Succesfully Saved
Transformation Pipeline and Model Succesfully Saved
Transformation Pipeline and Model Succesfully Saved
Transformation Pipeline and Model Succesfully Saved
